In [1]:
import sys
sys.path.append('../../robotics-course/build')
sys.path.append('../../robotics-course/scenarios/build')
import cv2 as cv
import numpy as np
import libry as ry
import time
# from perception import find_ball
print(cv.__version__)

4.2.0


In [2]:
#Let's edit the real world before we create the simulation
RealWorld = ry.Config()
RealWorld.addFile("../../robotics-course/scenarios/challenge.g")
V = ry.ConfigurationViewer()
V.setConfiguration(RealWorld)

In [3]:
# you can also change the shape & size
targetObj=RealWorld.getFrame("obj0")
targetObj.setColor([1.,0,0])
# targetObj.setShape(ry.ST.sphere, [.03])
RealWorld.getFrame("obj0").setShape(ry.ST.ssBox, [.05, .05, .05, .01])
targetObj.setPosition([-0.1, .1, 0.7])
targetObj.setContact(1)

#remove some objects
for o in range(1,30):
    name = "obj%i" % o
    print("deleting", name)
    RealWorld.delFrame(name)

V.recopyMeshes(RealWorld)
V.setConfiguration(RealWorld)

deleting obj1
deleting obj2
deleting obj3
deleting obj4
deleting obj5
deleting obj6
deleting obj7
deleting obj8
deleting obj9
deleting obj10
deleting obj11
deleting obj12
deleting obj13
deleting obj14
deleting obj15
deleting obj16
deleting obj17
deleting obj18
deleting obj19
deleting obj20
deleting obj21
deleting obj22
deleting obj23
deleting obj24
deleting obj25
deleting obj26
deleting obj27
deleting obj28
deleting obj29


In [4]:
# instantiate the simulation
S = RealWorld.simulation(ry.SimulatorEngine.physx, True)
S.addSensor("camera")

In [5]:
# create your model world
C = ry.Config()
C.addFile('../../robotics-course/scenarios/pandasTable.g')
#V = ry.ConfigurationViewer()
V.setConfiguration(C)
cameraFrame = C.frame("camera")

#the focal length
f = 0.895
f = f * 360.
fxfypxpy = [f, f, 320., 180.]

# save initial background
[rgb0, depth0] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
points0 = S.depthData2pointCloud(depth0, fxfypxpy)

In [6]:
obj = C.addFrame("object")
# obj.setShape(ry.ST.sphere, [.03])
obj.setShape(ry.ST.ssBox, [.05, .05, .05, .01])
obj.setColor([0,1,0])
obj.setContact(1)

V.setConfiguration(C)

In [ ]:
points = []
tau = .01


APPORACHING_OBJECT = 0
CLOSING_GRIPPER = 1
GOING_BACK = 2
RELEASING_OBJECT = 3

RSTATE = APPORACHING_OBJECT

def update(komo):
    komo.optimize(True)
    end_config = komo.getConfiguration(0)
    C.setFrameState(end_config)
    #RealWorld.setFrameState(end_config)
    V.setConfiguration(C)
    #return C.evalFeature(ry.FS.qItself, [])
    return C.getJointState()

def go_to_object(gripper, obj):
    komo = C.komo_path(1.,1,tau,True)
    komo.addObjective([1.], ry.FS.positionDiff, [gripper,obj], ry.OT.sos, [1e3]);
    #komo.addObjective([1.], ry.FS.positionDiff, ["R_gripperCenter", "percept"], ry.OT.sos, [1e2]);
    komo.addObjective([1.], ry.FS.vectorZ, [gripper], ry.OT.sos, [1e2], target=[0,0,1]);
    komo.addObjective([1.], ry.FS.scalarProductXZ, [obj,gripper], ry.OT.sos, [1e2]);
    komo.addObjective([1.], ry.FS.scalarProductXY, [obj,gripper], ry.OT.sos, [1e2]);
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
    #komo.addObjective([1.], ry.FS.qItself, [], ry.OT.sos, [1e3], order=1);
    komo.addObjective([], ry.FS.qItself, ["R_finger1"], ry.OT.eq, [1e1], order=1)
    return komo

def move_to_position(pos):
    komo = C.komo_path(1.,1,tau,True)
    komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.ineq, [1e2])
    komo.addObjective([1.], ry.FS.position, ["R_gripperCenter"], ry.OT.sos, [1e3], target=pos);
    #komo.addObjective([], ry.FS.qItself, ["R_finger1"], ry.OT.eq, [1e1], order=1)
    return komo
    
def start_grip_object(gripper):
    if not S.getGripperIsGrasping(gripper):
        S.closeGripper(gripper)

def grip_is_finished(gripper):
    return S.getGripperIsGrasping(gripper)

is_gripping = False
fall_delay = 0

for t in range(10000):
    
    print("Iteration: {} State: {}".format(t, RSTATE))
    
    time.sleep(0.01)

    #grab sensor readings from the simulation
    q = S.get_q()
    
    
    if t%10 == 0:
        [rgb, depth] = S.getImageAndDepth()  #we don't need images with 100Hz, rendering is slow
        points = S.depthData2pointCloud(depth, fxfypxpy)
       
        # skipping perception
        p_obj = targetObj.getPosition()   
        r_obj = targetObj.getQuaternion()
        obj.setPosition(p_obj)
        obj.setQuaternion(r_obj)

        V.recopyMeshes(C)
        V.setConfiguration(C)
        
        
    if RSTATE == CLOSING_GRIPPER:
        if not is_gripping:
            start_grip_object("R_gripper")
            is_gripping = True
        elif grip_is_finished("R_gripper"):
            RSTATE = GOING_BACK
            is_gripping = False
        q = S.get_q()
    
    elif RSTATE == APPORACHING_OBJECT:
        komo = go_to_object("R_gripperCenter", "object")
        q = update(komo)
        
        diff,_ = C.evalFeature(ry.FS.positionDiff, ["R_gripperCenter","object"])
        print(np.linalg.norm(diff))
        if np.linalg.norm(diff) < 0.02:
            RSTATE = CLOSING_GRIPPER
        
    elif RSTATE == GOING_BACK:
        pos = [0,0,1]
        komo = move_to_position(pos)
        q = update(komo)
        
        diff,_ = C.evalFeature(ry.FS.position, ["R_gripperCenter"])
        diff -= np.array(pos)
        print(np.linalg.norm(diff))
        if np.linalg.norm(diff) < 0.1:
            RSTATE = RELEASING_OBJECT
    
    elif RSTATE == RELEASING_OBJECT:
        S.openGripper("R_gripper")
        fall_delay += 1
        if fall_delay > 50:
            RSTATE = APPORACHING_OBJECT
            fall_delay = 0
    
    S.step(q, tau, ry.ControlMode.position)


Iteration: 0 State: 0
0.6819101199365638
Iteration: 1 State: 0
0.5337233261584319
Iteration: 2 State: 0
0.46346188891123025
Iteration: 3 State: 0
0.42428594604186526
Iteration: 4 State: 0
0.39673619130558785
Iteration: 5 State: 0
0.3738343176541567
Iteration: 6 State: 0
0.35307045885184635
Iteration: 7 State: 0
0.33370781873094296
Iteration: 8 State: 0
0.3152097499183339
Iteration: 9 State: 0
0.29761380675288207
Iteration: 10 State: 0
0.2892504495898023
Iteration: 11 State: 0
0.2727499377658552
Iteration: 12 State: 0
0.25708844117319274
Iteration: 13 State: 0
0.24224192373280892
Iteration: 14 State: 0
0.22819998850879047
Iteration: 15 State: 0
0.21495661941738395
Iteration: 16 State: 0
0.20248641078902682
Iteration: 17 State: 0
0.19076237507556926
Iteration: 18 State: 0
0.17977278015716225
Iteration: 19 State: 0
0.16950651735202665
Iteration: 20 State: 0
0.1533163001534313
Iteration: 21 State: 0
0.144534688499328
Iteration: 22 State: 0
0.13638255929753745
Iteration: 23 State: 0
0.12873

0.12440453994704737
Iteration: 285 State: 2
0.11754596065208683
Iteration: 286 State: 2
0.11102039247404695
Iteration: 287 State: 2
0.10473980926077753
Iteration: 288 State: 2
0.09881649417718988
Iteration: 289 State: 3
Iteration: 290 State: 3
Iteration: 291 State: 3
Iteration: 292 State: 3
Iteration: 293 State: 3
Iteration: 294 State: 3
Iteration: 295 State: 3
Iteration: 296 State: 3
Iteration: 297 State: 3
Iteration: 298 State: 3
Iteration: 299 State: 3
Iteration: 300 State: 3
Iteration: 301 State: 3
Iteration: 302 State: 3
Iteration: 303 State: 3
Iteration: 304 State: 3
Iteration: 305 State: 3
Iteration: 306 State: 3
Iteration: 307 State: 3
Iteration: 308 State: 3
Iteration: 309 State: 3
Iteration: 310 State: 3
Iteration: 311 State: 3
Iteration: 312 State: 3
Iteration: 313 State: 3
Iteration: 314 State: 3
Iteration: 315 State: 3
Iteration: 316 State: 3
Iteration: 317 State: 3
Iteration: 318 State: 3
Iteration: 319 State: 3
Iteration: 320 State: 3
Iteration: 321 State: 3
Iteration: 3

Iteration: 610 State: 1
Iteration: 611 State: 1
Iteration: 612 State: 1
Iteration: 613 State: 1
Iteration: 614 State: 1
Iteration: 615 State: 1
Iteration: 616 State: 1
Iteration: 617 State: 1
Iteration: 618 State: 1
Iteration: 619 State: 1
Iteration: 620 State: 1
Iteration: 621 State: 1
Iteration: 622 State: 1
Iteration: 623 State: 1
Iteration: 624 State: 1
Iteration: 625 State: 1
Iteration: 626 State: 1
Iteration: 627 State: 1
Iteration: 628 State: 1
Iteration: 629 State: 1
Iteration: 630 State: 1
Iteration: 631 State: 1
Iteration: 632 State: 1
Iteration: 633 State: 1
Iteration: 634 State: 1
Iteration: 635 State: 1
Iteration: 636 State: 1
Iteration: 637 State: 1
Iteration: 638 State: 1
Iteration: 639 State: 1
Iteration: 640 State: 1
Iteration: 641 State: 1
Iteration: 642 State: 1
Iteration: 643 State: 1
Iteration: 644 State: 1
Iteration: 645 State: 1
Iteration: 646 State: 1
Iteration: 647 State: 1
Iteration: 648 State: 1
Iteration: 649 State: 1
Iteration: 650 State: 1
Iteration: 651 S

0.028294547310781317
Iteration: 845 State: 0
0.02822011363684336
Iteration: 846 State: 0
0.02814487714019129
Iteration: 847 State: 0
0.028071199257720024
Iteration: 848 State: 0
0.02799508196702791
Iteration: 849 State: 0
0.027920989524280276
Iteration: 850 State: 0
0.02784596420171426
Iteration: 851 State: 0
0.02777309029388727
Iteration: 852 State: 0
0.027698355291050792
Iteration: 853 State: 0
0.027625500447593373
Iteration: 854 State: 0
0.027550933941886124
Iteration: 855 State: 0
0.02747764558812372
Iteration: 856 State: 0
0.02740464739105397
Iteration: 857 State: 0
0.027332496502533794
Iteration: 858 State: 0
0.0272600779359384
Iteration: 859 State: 0
0.02718783009152976
Iteration: 860 State: 0
0.027115957706093954
Iteration: 861 State: 0
0.027044491285762064
Iteration: 862 State: 0
0.026972191489832098
Iteration: 863 State: 0
0.02690193296762806
Iteration: 864 State: 0
0.026831520634198594
Iteration: 865 State: 0
0.026761382417208265
Iteration: 866 State: 0
0.026690809791448732


Iteration: 1053 State: 1
Iteration: 1054 State: 1
Iteration: 1055 State: 1
Iteration: 1056 State: 1
Iteration: 1057 State: 1
Iteration: 1058 State: 1
Iteration: 1059 State: 1
Iteration: 1060 State: 1
Iteration: 1061 State: 1
Iteration: 1062 State: 1
Iteration: 1063 State: 1
Iteration: 1064 State: 1
Iteration: 1065 State: 1
Iteration: 1066 State: 1
Iteration: 1067 State: 1
Iteration: 1068 State: 1
Iteration: 1069 State: 1
Iteration: 1070 State: 1
Iteration: 1071 State: 1
Iteration: 1072 State: 1
Iteration: 1073 State: 1
Iteration: 1074 State: 1
Iteration: 1075 State: 1
Iteration: 1076 State: 1
Iteration: 1077 State: 1
Iteration: 1078 State: 1
Iteration: 1079 State: 1
Iteration: 1080 State: 1
Iteration: 1081 State: 1
Iteration: 1082 State: 1
Iteration: 1083 State: 1
Iteration: 1084 State: 1
Iteration: 1085 State: 1
Iteration: 1086 State: 1
Iteration: 1087 State: 1
Iteration: 1088 State: 1
Iteration: 1089 State: 1
Iteration: 1090 State: 1
Iteration: 1091 State: 1
Iteration: 1092 State: 1


...

Iteration: 1991 State: 1
Iteration: 1992 State: 1
Iteration: 1993 State: 1
Iteration: 1994 State: 1
Iteration: 1995 State: 1
Iteration: 1996 State: 1
Iteration: 1997 State: 1
Iteration: 1998 State: 1
Iteration: 1999 State: 1
Iteration: 2000 State: 1
Iteration: 2001 State: 1
Iteration: 2002 State: 1
Iteration: 2003 State: 1
Iteration: 2004 State: 1
Iteration: 2005 State: 1
Iteration: 2006 State: 1
Iteration: 2007 State: 1
Iteration: 2008 State: 1
Iteration: 2009 State: 1
Iteration: 2010 State: 1
Iteration: 2011 State: 1
Iteration: 2012 State: 1
Iteration: 2013 State: 1
Iteration: 2014 State: 1
Iteration: 2015 State: 1
Iteration: 2016 State: 1
Iteration: 2017 State: 1
Iteration: 2018 State: 1
Iteration: 2019 State: 1
Iteration: 2020 State: 1
Iteration: 2021 State: 1
Iteration: 2022 State: 1
Iteration: 2023 State: 1
Iteration: 2024 State: 1
Iteration: 2025 State: 1
Iteration: 2026 State: 1
Iteration: 2027 State: 1
Iteration: 2028 State: 1
Iteration: 2029 State: 1
Iteration: 2030 State: 1


Iteration: 2331 State: 1
Iteration: 2332 State: 1
Iteration: 2333 State: 1
Iteration: 2334 State: 1
Iteration: 2335 State: 1
Iteration: 2336 State: 1
Iteration: 2337 State: 1
Iteration: 2338 State: 1
Iteration: 2339 State: 1
Iteration: 2340 State: 1
Iteration: 2341 State: 1
Iteration: 2342 State: 1
Iteration: 2343 State: 1
Iteration: 2344 State: 1
Iteration: 2345 State: 1
Iteration: 2346 State: 1
Iteration: 2347 State: 1
Iteration: 2348 State: 1
Iteration: 2349 State: 1
Iteration: 2350 State: 1
Iteration: 2351 State: 1
Iteration: 2352 State: 1
Iteration: 2353 State: 1
Iteration: 2354 State: 1
Iteration: 2355 State: 1
Iteration: 2356 State: 1
Iteration: 2357 State: 1
Iteration: 2358 State: 1
Iteration: 2359 State: 1
Iteration: 2360 State: 1
Iteration: 2361 State: 1
Iteration: 2362 State: 1
Iteration: 2363 State: 1
Iteration: 2364 State: 1
Iteration: 2365 State: 1
Iteration: 2366 State: 1
Iteration: 2367 State: 1
Iteration: 2368 State: 1
Iteration: 2369 State: 1
Iteration: 2370 State: 1


0.023883958399663008
Iteration: 2577 State: 0
0.023762796466237506
Iteration: 2578 State: 0
0.023597101118609704
Iteration: 2579 State: 0
0.023518922052285868
Iteration: 2580 State: 0
0.023062654151446087
Iteration: 2581 State: 0
0.023170535071446435
Iteration: 2582 State: 0
0.022553096142028696
Iteration: 2583 State: 0
0.02265514458761309
Iteration: 2584 State: 0
0.022646728664234755
Iteration: 2585 State: 0
0.02257419591222462
Iteration: 2586 State: 0
0.02251166484766641
Iteration: 2587 State: 0
0.02243503630092552
Iteration: 2588 State: 0
0.022313418062607893
Iteration: 2589 State: 0
0.022138115757347806
Iteration: 2590 State: 0
0.022037338989642732
Iteration: 2591 State: 0
0.021557798820905996
Iteration: 2592 State: 0
0.021168709831910406
Iteration: 2593 State: 0
0.021290278796288132
Iteration: 2594 State: 0
0.02129466308751894
Iteration: 2595 State: 0
0.02121985924082752
Iteration: 2596 State: 0
0.02119867651483444
Iteration: 2597 State: 0
0.02112849802700304
Iteration: 2598 State

Iteration: 2889 State: 1
Iteration: 2890 State: 1
Iteration: 2891 State: 1
Iteration: 2892 State: 1
Iteration: 2893 State: 1
Iteration: 2894 State: 1
Iteration: 2895 State: 1
Iteration: 2896 State: 1
Iteration: 2897 State: 1
Iteration: 2898 State: 1
Iteration: 2899 State: 1
Iteration: 2900 State: 1
Iteration: 2901 State: 1
Iteration: 2902 State: 1
Iteration: 2903 State: 1
Iteration: 2904 State: 1
Iteration: 2905 State: 1
Iteration: 2906 State: 1
Iteration: 2907 State: 1
Iteration: 2908 State: 1
Iteration: 2909 State: 1
Iteration: 2910 State: 1
Iteration: 2911 State: 1
Iteration: 2912 State: 1
Iteration: 2913 State: 1
Iteration: 2914 State: 1
Iteration: 2915 State: 1
Iteration: 2916 State: 1
Iteration: 2917 State: 1
Iteration: 2918 State: 1
Iteration: 2919 State: 1
Iteration: 2920 State: 1
Iteration: 2921 State: 1
Iteration: 2922 State: 1
Iteration: 2923 State: 1
Iteration: 2924 State: 1
Iteration: 2925 State: 1
Iteration: 2926 State: 1
Iteration: 2927 State: 1
Iteration: 2928 State: 1


Iteration: 3228 State: 1
Iteration: 3229 State: 1
Iteration: 3230 State: 1
Iteration: 3231 State: 1
Iteration: 3232 State: 1
Iteration: 3233 State: 1
Iteration: 3234 State: 1
Iteration: 3235 State: 1
Iteration: 3236 State: 1
Iteration: 3237 State: 1
Iteration: 3238 State: 1
Iteration: 3239 State: 1
Iteration: 3240 State: 1
Iteration: 3241 State: 1
Iteration: 3242 State: 1
Iteration: 3243 State: 1
Iteration: 3244 State: 1
Iteration: 3245 State: 1
Iteration: 3246 State: 1
Iteration: 3247 State: 1
Iteration: 3248 State: 1
Iteration: 3249 State: 1
Iteration: 3250 State: 1
Iteration: 3251 State: 1
Iteration: 3252 State: 1
Iteration: 3253 State: 1
Iteration: 3254 State: 1
Iteration: 3255 State: 1
Iteration: 3256 State: 1
Iteration: 3257 State: 1
Iteration: 3258 State: 1
Iteration: 3259 State: 1
Iteration: 3260 State: 1
Iteration: 3261 State: 1
Iteration: 3262 State: 1
Iteration: 3263 State: 1
Iteration: 3264 State: 1
Iteration: 3265 State: 1
Iteration: 3266 State: 1
Iteration: 3267 State: 1
